In [21]:
import pandas as pd
import numpy as np
import math as m

In [22]:
def calc_poisson(Lambda, n_corners):
    """
    Функция, считающая значения пуассоновского распределения
    для заданной лямбды и кол-ва событий (угловых).
    """
    return (Lambda**n_corners / m.factorial(n_corners)) * m.exp(-Lambda)

In [23]:
def create_poisson_matrix(Lh, La, size):
    """
    Функция, заполняющая матрицу пуассоновского распределения 
    при условии наступления i событий у Away и j событий у Home.
    Использую версию функции из BR_bands без H и A
    Lh - лямбда home
    La - лямбда away
    size - размер квадратной матрицы
    """
    m = np.zeros((size, size))
    for h_corner in range(size):
        for a_corner in range(size):
            # условная вероятность - унможаем вероятность события home на вер-ть события away
            m[a_corner][h_corner] = calc_poisson(Lh, h_corner)*calc_poisson(La, a_corner)
    return m

In [24]:
def create_prob_dict(p_matrix, mode='Home'):
    """
    Функция - считает суммарные вероятности <=N событий для Home (столбцы) или Away (строки),
    Заносит их в словарь линии от 0.5 до 8.5
    p_matrix - посчитанная матрица вероятностей
    mode - по какому направлению суммируем, 'Home' - столбцы, 'Away' - строки
    """
    # считаем сумму, получаем массив размера size матрицы
    if mode == 'Home':
        col_sum = np.sum(p_matrix, axis=0)
    elif mode == 'Away':
        col_sum = np.sum(p_matrix, axis=1)
    # т.к. у нас under probability, по полученному массиму считаем кумулятивную сумму
    col_sum = np.cumsum(col_sum)
    
    # формируем словарь с under probability
    p_dict = {}
    i = 0
    for i in range(9):
        p_dict[i+0.5] = round(col_sum[i],3)
    return p_dict

In [25]:
# пример работы функции 
p_matrix = create_poisson_matrix(Lh = 3, La = 4, size = 20)
create_prob_dict(p_matrix, mode='Home')

{0.5: 0.05,
 1.5: 0.199,
 2.5: 0.423,
 3.5: 0.647,
 4.5: 0.815,
 5.5: 0.916,
 6.5: 0.966,
 7.5: 0.988,
 8.5: 0.996}

In [36]:
def find_lh(H=0, Line_h=3.5, Line_a=3.5, U_h=1.85, O_h=1.85):
    """
    Функция подибрает Lh из заданных вариантов
    H - текущее кол-во угловых Home
    Line_h - линия home
    U_h - коэффициент under home
    O_h - коэффициент over home
    значения по умолчанию равны значениям по умолчанию в excel - для простоты тестирования
    """
    # рассчитываем стартовые параметры
    k_h = round(1 / (U_h * (1/U_h + 1/O_h)),3) # under probability home

    Lh = Line_h # первая прикидка Lh для расчета
    La = Line_a # первая прикидка La для расчета

    Line_h = Line_h - H  # Сергей - наверное, надо как-то обрабатывать случаи, когда Line_h < H ? 
    Delta = Line_h
    
    for j in range(1,201): # j - параметр для подбора Lh
        # предполагаем, что La = 0, строим матрицу и считаем суммарные вер-ти 
        p_matrix = create_poisson_matrix(Lh, La, 21)
        p_dict = create_prob_dict(p_matrix, mode='Home')
        
        # надо подобрать такую Lh, чтобы суммарная вероятность в линии = k_h
        # если такой Lh не находится - мы меняем Lh на Delta/j и повторяем процесс заново 
        # (расчет матрицы для нового Lh, суммарные вероятности, проверка на равенство k_h )
        for key in p_dict:
            if key == Line_h:
                if p_dict[key] == k_h:
                    return Lh
                elif p_dict[key] < k_h:
                    Lh = Lh - Delta/j
                elif p_dict[key] > k_h:
                    Lh = Lh + Delta/j
    return Lh

In [38]:
def find_la(Lh, A=0, Line_a=3.5, U_a=1.85, O_a=1.85):
    """
    Функция подибрает La из заданных вариантов
    Lh - подобранный Lh из find_lh 
    A - текущее кол-во угловых Away
    Line_a - линия home
    U_a - коэффициент under home
    O_a - коэффициент over home
    значения по умолчанию равны значениям по умолчанию в excel - для простоты тестирования
    """
    # рассчитываем стартовые параметры
    k_a = round(1 / (U_a * (1/U_a + 1/O_a)),3) # under probability away

    La = Line_a # первая прикидка La для расчета
    Line_a = Line_a - A  # Сергей - наверное, надо как-то обрабатывать случаи, когда Line_a < A ? 
    Delta = Line_a
    
    for j in range(1,201): # j - параметр для подбора La
        # предполагаем, что La = 0, строим матрицу и считаем суммарные вер-ти 
        p_matrix = create_poisson_matrix(Lh, La, 21)
        p_dict = create_prob_dict(p_matrix, mode='Away')
        
        # надо подобрать такую La, чтобы суммарная вероятность в линии = k_a
        # если такой La не находится - мы меняем La на Delta/j и повторяем процесс заново 
        # (расчет матрицы для нового La, суммарные вероятности, проверка на равенство k_a )
        for key in p_dict:
            if key == Line_a:
                if p_dict[key] == k_a:
                    return La
                elif p_dict[key] < k_a:
                    La = La - Delta/j
                elif p_dict[key] > k_a:
                    La = La + Delta/j
    return La

In [40]:
# Тест 1
Lh = find_lh(H=1, )
La = find_la(Lh)
print(Lh,'\n',La)

3.674033244447331 
 3.674033244447331


In [41]:
# Тест 2
Lh = find_lh(H=1, Line_h=4.5, Line_a=4.5, U_h=2.5, O_h=2.5)
La = find_la(Lh, A=0, Line_a=4.5, U_a=3.5, O_a=3.5)
print(Lh,'\n',La)

3.6719907607872 
 4.673371315628099
